In [1]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
# fix randow seed for reporducibility
seed =7
np.random.seed(seed)

In [3]:
#load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
# - Convolutional input layer, 32 feature maps with a size of 3×3 and a rectifier activation function.
# - Dropout layer at 20%.
# - Convolutional layer, 32 feature maps with a size of 3×3 and a rectifier activation function.
# - Max Pool layer with size 2×2.
# - Convolutional layer, 64 feature maps with a size of 3×3 and a rectifier activation function.
# - Dropout layer at 20%.
# - Convolutional layer, 64 feature maps with a size of 3×3 and a rectifier activation function.
# - Max Pool layer with size 2×2.
# - Convolutional layer, 128 feature maps with a size of 3×3 and a rectifier activation function.
# - Dropout layer at 20%.
# - Convolutional layer,128 feature maps with a size of 3×3 and a rectifier activation function.
# - Max Pool layer with size 2×2.
# - Flatten layer.
# - Dropout layer at 20%.
# - Fully connected layer with 1024 units and a rectifier activation function.
# - Dropout layer at 20%.
# - Fully connected layer with 512 units and a rectifier activation function.
# - Dropout layer at 20%.
# - Fully connected output layer with 10 units and a softmax activation function.

# create the model
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(3,32,32), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
# compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 16, 16)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 16, 16)        36928     
__________

In [8]:
np.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 1369s - loss: 1.9254 - acc: 0.2943 - val_loss: 1.6915 - val_acc: 0.3976
Epoch 2/25
50000/50000 [==============================] - 1361s - loss: 1.5185 - acc: 0.4451 - val_loss: 1.3520 - val_acc: 0.5144
Epoch 3/25
50000/50000 [==============================] - 1446s - loss: 1.3319 - acc: 0.5184 - val_loss: 1.2934 - val_acc: 0.5334
Epoch 4/25
50000/50000 [==============================] - 1434s - loss: 1.1962 - acc: 0.5739 - val_loss: 1.1274 - val_acc: 0.5970
Epoch 5/25
50000/50000 [==============================] - 1441s - loss: 1.0805 - acc: 0.6129 - val_loss: 1.0731 - val_acc: 0.6147
Epoch 6/25
50000/50000 [==============================] - 1484s - loss: 0.9928 - acc: 0.6478 - val_loss: 0.9305 - val_acc: 0.6738
Epoch 7/25
50000/50000 [==============================] - 1497s - loss: 0.9147 - acc: 0.6745 - val_loss: 0.8455 - val_acc: 0.6972
Epoch 8/25
38400/50000 [================

KeyboardInterrupt: 

In [ ]:
#http://machinelearningmastery.com/object-recognition-convolutional-neural-networks-keras-deep-learning-library/